<h1> <center> Альтернативные нейросетевые модели <br> для последовательных данных </center></h1>

In [30]:
import torch
from torch import nn
from torch.autograd import Variable
import data

Датасет Penn Treebank

In [27]:
corpus = data.Corpus('./data')
sequence_length = 35

In [18]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    if torch.cuda.is_available():
        data = data.cuda()
    return data

In [19]:
eval_batch_size = 10
batch_size = 20
train_data = batchify(corpus.train, batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

/home/irina/anaconda3/envs/dl/lib/python3.6/site-packages/torch/cuda/__init__.py:97: UserWarning: 
    Found GPU0 GeForce GT 620M which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [28]:
def get_batch(source, i, evaluation=False):
    seq_len = min(sequence_length, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    return data, target

In [31]:
data, targets = get_batch(train_data, 0)

In [44]:
corpus.dictionary.idx2word[1]

'banknote'

## ByteNet
Нейросеть для перевода и language modeling. Inspired by WaveNet. 
Основные две идеи: 
* стакнуть декодер сверху энкодера (то есть выход энколера является входом декодера). Мотивация: повысить пропускную способность того, что получаем после энкодера в декодер.
* использовать dynamic unfolding (динамическое разворачивание): то есть размер выхода энкодера определяется как линейная функция от входа. Мотивация: решить проблему разницы длин последовательностей входа и выхода.
Архитектура: похоже на WaveNet и PixelCNN. Декодер: masked-conv (чтобы не заглядывать в будущее), которая ещё и dilated, обёрнуто в residial block. Энкодер такой же, только без masked. 
Для NMT: энкодер+декодер, для language modeling только decoder. Оба таска делаются by chars.
Основное преимущество: линейность времени работы (по длине последовательности).